In [1]:
import os
import sys
import pandas as pd
from google import genai
from tqdm.auto import tqdm
from dotenv import load_dotenv

In [6]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

load_dotenv()
client = genai.Client()


In [3]:
data_path = '../data/df_cleaned.parquet'
try:
    df = pd.read_parquet(data_path)
    print(f"Данные успешно загружены из {data_path}. Количество записей: {len(df)}")
except FileNotFoundError:
    print(f"Ошибка: Файл {data_path} не найден. Убедитесь, что вы запустили ноутбук с EDA и предобработкой.")

Данные успешно загружены из ../data/df_cleaned.parquet. Количество записей: 19276


In [4]:
abstract_list = df['abstract'].tolist()

batch_size = 500
total_tokens = 0

In [7]:
for i in tqdm(range(0, len(abstract_list), batch_size), desc="Подсчет токенов"):
    batch = abstract_list[i:i + batch_size]
    response = client.models.count_tokens(
    model="gemini-2.0-flash", contents=batch
    )
    total_tokens += response.total_tokens
    
avg_tokens_per_abstract = total_tokens / len(df) if len(df) > 0 else 0

Подсчет токенов:   0%|          | 0/39 [00:00<?, ?it/s]

In [8]:
print("\n--- Результаты анализа токенов ---")
print(f"Общее количество токенов во всех аннотациях: {total_tokens:,}")
print(f"Среднее количество токенов на одну аннотацию: {avg_tokens_per_abstract:.2f}")


--- Результаты анализа токенов ---
Общее количество токенов во всех аннотациях: 3,660,701
Среднее количество токенов на одну аннотацию: 189.91


Датасет содержит 19 000 аннотаций с общим объемом текста более 3.5 миллионов токенов. Это означает, что **прямая обработка всего корпуса с помощью LLM API является ресурсоемкой задачей, требующей оптимизации как по стоимости, так и по времени выполнения.**
Для извлечения ключевых фраз с помощью LLM я рассмотрел две основные стратегии:
> 1.  **Прямая генерация (Zero-Shot/Few-Shot Prompting):** Подача аннотации напрямую в LLM с инструкцией извлечь ключевые фразы.
> 2.  **Двухэтапный пайплайн (Candidate Generation & Filtering):** Использование классического алгоритма (например, YAKE) для генерации большого списка фраз-кандидатов с последующей фильтрацией и ранжированием этого списка с помощью LLM."
* Я выбрал модель семейства Gemini Flash 2.5 Lite от Google с доступом по API. Главные преимущества моделей Gemini: большое контекстное окно, одна из самых быстрых генераций, недорогостоящие и дополнительно поддерживают Structured Output, что позволит уменьшить риски генерации неверных JSON и количество токенов в системном промпте.

Обработка 19 000 документов с помощью LLM API требует инженерного подхода к оптимизации, так как наивное решение "один документ — один запрос" неэффективно по трем ключевым параметрам:

 1. Отправка 19 000 запросов упирается в ограничения API (15 RPM), что растягивает обработку на десятки часов.
 2. Накладные расходы на каждый из 19 000 HTTP-запросов делают процесс недопустимо медленным.
 3. При индивидуальных запросах системный промпт (≈100 токенов) отправляется с каждым документом. Это приводит к избыточной оплате **почти 1.9 миллиона токенов** только за повторение одних и тех же инструкций.

 Для решения этих проблем я реализую **батчинг**. Несколько документов (например, 20-30) и их кандидаты объединяются в один API-запрос со сложным структурированным промптом. Этот подход позволяет:
 *   **Радикально снизить стоимость** за счет однократной отправки инструкций на целый батч.
 *   **Уложиться в Rate Limits** благодаря сокращению количества запросов на порядки.
 *   **Ускорить общую обработку** за счет минимизации сетевых задержек.

 Для демонстрации работоспособности этого масштабируемого пайплайна, вычисления будут произведены на репрезентативной выборке из 200 аннотаций.

